This notebook will have an attempt to create an agent that uses both sql and a vector database to answer user questions within scope, and if it couldn't answer the question.

In [ ]:
# Load environment and basic libs
from dotenv import load_dotenv
load_dotenv()

import os
import json
from typing import Dict, Any, List

print('Environment loaded')

Environment loaded


In [ ]:
# Imports for LangChain, Ollama, FAISS, and SQL
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import CSVLoader

# SQL/alchemy
from sqlalchemy import create_engine, inspect, text
import pandas as pd

# Utilities
from base64 import b64decode

# Setup Ollama endpoints and models (read from env or use sensible defaults)
OLLAMA_BASE_URL = os.getenv('OLLAMA_BASE_URL')
CHAT_MODEL = os.getenv('OLLAMA_CHAT_MODEL')
EMBEDDING_MODEL = os.getenv('OLLAMA_EMBEDDING_MODEL')

chat_model = ChatOllama(base_url=OLLAMA_BASE_URL, model=CHAT_MODEL)
embeddings = OllamaEmbeddings(base_url=OLLAMA_BASE_URL, model=EMBEDDING_MODEL)

print('Models initialized:', CHAT_MODEL, EMBEDDING_MODEL)

Models initialized: llava:latest embeddinggemma:latest


In [ ]:
# Load Chroma vectorstore
try:
# Creating the vector database
    vectorstore = Chroma(collection_name="multi_modal_rag", embedding_function=embeddings, persist_directory='../data/chroma_db')
    retriever = vectorstore.as_retriever()
    print('Loaded Chroma vectorstore and created retriever')
except Exception as e:
    vectorstore = None
    retriever = None
    print('Could not load Chroma vectorstore:', e)

Loaded Chroma vectorstore and created retriever


In [ ]:
from langchain_community.utilities.sql_database import SQLDatabase

db_uri = f"mssql+pyodbc://zaid-allawanseh/InsuranceNetwork?driver=ODBC+Driver+17+for+SQL+Server"
db = SQLDatabase.from_uri(db_uri, schema='dbo', engine_args={'isolation_level':'READ COMMITTED'})


In [ ]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers.string import StrOutputParser

def get_schema(_):
    return db.get_table_info()

def run_query(query):
    print('Running this query', query)
    result = db.run(query, execution_options={'isolation_level':'READ COMMITTED'})
    print(result)
    return result 

def write_sql_query(llm):
    template = """write a T-SQL query that would answer the question below based on the schema provided do NOT explain what you do. no pre-amble. Do not include any Markdown code fences (, sql, etc.). Only return the raw SQL text. Keep in mind that the database is pure text, so use the LIKE keyword instead of the equal sign, also all of the data is in Arabic, so Add 'N' when you want to query
    {schema}
    
    Question: {question}"""

    prompt = ChatPromptTemplate.from_messages(
        [
            ('system', '''given an input question, convert it to a T-SQL query, do NOT return anything except the query, do NOT explain what you do, no pre-amble. Do not include any Markdown code fences (, sql, etc.). Only return the raw SQL text.'''),
            ('human', template)
        ]
    )
    return (
        RunnablePassthrough.assign(schema=get_schema)
        | prompt
        | llm
        | StrOutputParser()
        )

In [ ]:
def answer_query(query, llm):
    template = '''Based on the table schema below, questoin, sql query and the sql response, write a natural language response, no pre-amble.
    {schema}
    
    Questoin: {question}
    SQL Query: {query}
    SQL Response: {response}'''

    prompt_reponse = ChatPromptTemplate.from_messages(
        [
            ('system',
             'given an input and a sql response, convert it to a natural language answer, if there were no answer then feel free to tell the user so.'),
             ('human',
              template)
        ]
    )

    full_chain = (
        RunnablePassthrough.assign(query=write_sql_query(llm))
        | RunnablePassthrough.assign(
            schema=get_schema,
            response=lambda x: run_query(x['query']))
        | prompt_reponse
        | llm
        | StrOutputParser()
        )
    return full_chain.invoke({'question': query})


In [ ]:
from langchain_groq import ChatGroq
import os

llm = ChatGroq(api_key=os.getenv('GROQ_API_KEY'), model='llama-3.3-70b-versatile') 

def sql_chain(query:str):    
    
    forbidden_statements = ['insert', 'update', 'delete', 'create', 'drop', 'alter']
    if any(stmt in query.lower() for stmt in forbidden_statements):
        raise ValueError("Please ask the agent for queries and not to modify the database.")
    return answer_query(query, llm)

# Router AI

In [ ]:
from langchain_core.prompts import PromptTemplate

router_prompt = PromptTemplate.from_template("""
You are a router between two systems:
1. Text-to-SQL: Use for questions that include something about the medical network.
2. RAG: Use for open-ended or descriptive questions.

Decide which one should handle the query.
Return only 'sql' or 'rag'.

Query: {query}
Decision:
""")

def decide_route(query):
    decision = llm.invoke(router_prompt.format(query=query)).content.strip().lower()
    return decision


In [ ]:
import sys
sys.path.append('../src')

In [ ]:
from chat import chain_with_sources as rag_chain

user_query = 'بدي رقم تلفون دكتور اويس'

decision = decide_route(user_query)

if decision == "sql":
    response = sql_chain(user_query)
else:
    response = rag_chain.invoke(user_query)
print(response)

Running this query SELECT [Phone] FROM dbo.[FullNetwork] WHERE [Name] LIKE N'%اويس%' AND [Main Category] LIKE N'%أطباء%'
[('0797113535',)]
رقم تلفون دكتور اويس هو 0797113535.
